In [10]:
import requests

from tqdm import tqdm
from bs4 import BeautifulSoup


In [11]:
def find_best_plural_form(word, suffix):
    len_w = len(word)
    len_s = len(suffix)
    best_offset = len_w
    best_offset_score = 1
    for offset in range(max(0, len_w - len_s), len_w):
        offset_score = sum([word[offset+i] == suffix[i] for i in range(min(len_s, len_w-offset))])
        if offset_score > best_offset_score:
            best_offset = offset
            best_offset_score = offset_score
    return 'die '+ word[:best_offset] + suffix

In [49]:
def parse_noun(word_romhead, word_h2):
    translation = ' '.join([
        a.text for a in word_romhead.find(class_='target').findAll('a', recursive=False)
    ]).replace('  ', ' ')
    if "r.m." in word_h2.text:
        article = "der"
    elif "r.ż." in word_h2.text:
        article = "die"
    else:
        article = "das"
    word_div = [
        div for div in word_romhead.findAll(class_='translations')
    ][0]
    word = ' '.join([
        a.text for a in word_div.findAll(
            class_='source'
        )[0].findAll(['a', 'strong'])
    ])
    flexion_text = word_h2.find(class_='flexion').text
    plural_suffix = flexion_text[flexion_text.index(',')+2:-1]
    if plural_suffix[0] == '‑':
        plural_form = find_best_plural_form(word, plural_suffix[1:])
    elif 'bez l.mn.' in plural_suffix:
        plural_form = '-'
    else:
        plural_form = 'die ' + plural_suffix
    zwroty_divs = [
        div for div
        in word_romhead.findAll(class_='translations')
        if 'zwroty' in div.find('h3').text
    ]
    if len(zwroty_divs) == 0:
        return translation, f"{article} {word}, {plural_form}\n"
    else:
        zwrot = ' '.join([
            a.text for a in zwroty_divs[0].findAll(
                class_='source'
            )[0].findAll('span')[0].findAll(['a', 'strong'])
        ])
        zwrot = zwrot.replace('  ', ' ')

        zwrot_translation = ' '.join([
            a.text for a in zwroty_divs[0].findAll(
                class_='target'
            )[0].findAll('a')
        ])
        zwrot_translation = zwrot_translation.replace('  ', ' ')
        return translation, f"{article} {word}, {plural_form}\n\n{zwrot}\n({zwrot_translation})\n"


def parse_verb(word_romhead, word_h2):
    translation = ' '.join([a.text for a in word_romhead.find(class_='target').findAll('a', recursive=False)]).replace('  ', ' ')
    word_div = [
        div for div in word_romhead.findAll(class_='translations')
    ][0]
    word = ' '.join([
        a.text for a in word_div.findAll(
            class_='source'
        )[0].findAll(['a', 'strong'])
    ])
    return translation, word

In [13]:
def download_page_as_word_romheads(word):
    url = f"https://pl.pons.com/t%C5%82umaczenie/niemiecki-polski/{word}"
    try:
        page = requests.get(url)
    except:
        print(f"Unable to get {word}, site {url} doesn't respond")
        return None, None
    soup = BeautifulSoup(page.content, "html.parser")
    word_romheads = soup.findAll(class_="rom first")
    return word_romheads

Debug below

In [40]:
word = ' '.join([
    a.text for a in word_div.findAll(
        class_='source'
    )[0].findAll(['a', 'strong'])
])

In [39]:
word

''

In [48]:
word_romhead = word_romheads[1]
word_h2 = word_romhead.find("h2")
translation = ' '.join([a.text for a in word_romhead.find(class_='target').findAll('a', recursive=False)]).replace('  ', ' ')
word = ' '.join([
    a.text for a in word_div.findAll(
        class_='source'
    )[0].findAll(['a', 'strong'])
])

AttributeError: 'NoneType' object has no attribute 'findAll'

In [41]:
word_romhead = word_romheads[0]
word_h2 = word_romhead.find("h2")
translation = ' '.join([
        a.text for a in word_romhead.find(class_='target').findAll('a', recursive=False)
    ]).replace('  ', ' ')
if "r.m." in word_h2.text:
    article = "der"
elif "r.ż." in word_h2.text:
    article = "die"
else:
    article = "das"
word_div = [
    div for div in word_romhead.findAll(class_='translations')
][0]
word = ' '.join([
    a.text for a in word_div.findAll(
        class_='source'
    )[0].findAll(['a', 'strong'])
])
flexion_text = word_h2.find(class_='flexion').text
plural_suffix = flexion_text[flexion_text.index(',')+2:-1]
if plural_suffix[0] == '‑':
    plural_form = find_best_plural_form(word, plural_suffix[1:])
elif 'bez l.mn.' in plural_suffix:
    plural_form = '-'
else:
    plural_form = 'die ' + plural_suffix
zwroty_divs = [
    div for div
    in word_romhead.findAll(class_='translations')
    if 'zwroty' in div.find('h3').text
]
if len(zwroty_divs) == 0:
    return translation, f"{article} {word}, {plural_form}\n"
else:
    zwrot = ' '.join([
        a.text for a in zwroty_divs[0].findAll(
            class_='source'
        )[0].findAll('span')[0].findAll(['a', 'strong'])
    ])
    zwrot = zwrot.replace('  ', ' ')

    zwrot_translation = ' '.join([
        a.text for a in zwroty_divs[0].findAll(
            class_='target'
        )[0].findAll('a')
    ])
    zwrot_translation = zwrot_translation.replace('  ', ' ')
    return translation, f"{article} {word}, {plural_form}\n\n{zwrot}\n({zwrot_translation})\n"


AttributeError: 'NoneType' object has no attribute 'text'

End of debug

In [43]:
def create_card(word_romheads):
    questions = []
    answers = []

    for word_romhead in word_romheads:
        if "ytkownika" in word_romhead.text:
            # skip user-created translations
            continue
        word_h2 = word_romhead.find("h2")
        word_class = word_h2.find(class_='wordclass').text
        if 'CZ.' in word_class:
            question, answer = parse_verb(word_romhead, word_h2)
        elif 'RZ.' in word_class:
            question, answer = parse_noun(word_romhead, word_h2)
        else:
            continue
        questions.append(question)
        answers.append(answer)
        if len(questions) > 1:
            break

    front = ''
    reverse = ''

    for i, q in enumerate(questions):
        front += f'{i+1}: {q}\n'

    for i, a in enumerate(answers):
        reverse += f'{a}\n\n'
    
    return front, reverse

In [45]:
word_romheads = download_page_as_word_romheads('frish')

In [50]:
front, reverse = create_card(word_romheads)
print(front)
print('-----')
print(reverse)
print('_________')

AttributeError: 'NoneType' object has no attribute 'findAll'

In [20]:
fronts = []
reverses = []
words = ['Katze', 'Land', 'Tochter', 'Lebensraum', 'Korkenzieher']
# words = ['Lebensraum']
for word in tqdm(words):
    front, reverse = create_card(word)
    if front is not None:
        fronts.append(front)
        reverses.append(reverse)

100%|██████████| 5/5 [00:07<00:00,  1.43s/it]


In [19]:
for front, reverse in zip(fronts, reverses):
    print(front)
    print('-----')
    print(reverse)
    print('_________')

1: przestrzeń życiowa

-----
der Le̱bensraum, die Le̱bensräume



_________


In [21]:
import csv
with open('eggs.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for f, r in zip(fronts, reverses):
        spamwriter.writerow([f, r])